In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim 
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, random_split
import gurobipy as gp
from gurobipy import GRB
from tqdm import tqdm 
from helper import *  

In [2]:
# Define the resnet model
class BasicBlock(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        stride: int = 1,
        expansion: int = 1,
        downsample: nn.Module = None
    ) -> None:
        super(BasicBlock, self).__init__()
        self.expansion = expansion
        self.downsample = downsample
        self.conv1 = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(
            out_channels,
            out_channels*self.expansion,
            kernel_size=3,
            padding=1,
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(out_channels*self.expansion)
    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return  out

In [3]:
# Define the resnet model 
class ResNet(nn.Module):
    def __init__(
        self,
        img_channels: int,
        num_layers: int,
        block: Type[BasicBlock],
        num_classes: int  = 2
    ) -> None:
        super(ResNet, self).__init__()
        if num_layers == 18:
            layers = [2, 2, 2, 2]
            self.expansion = 1

        self.in_channels = 64
        self.conv1 = nn.Conv2d(
            in_channels=img_channels,
            out_channels=self.in_channels,
            kernel_size=7,
            stride=2,
            padding=3,
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512*self.expansion, num_classes)
    def _make_layer(
        self,
        block: Type[BasicBlock],
        out_channels: int,
        blocks: int,
        stride: int = 1
    ) -> nn.Sequential:
        downsample = None
        if stride != 1:
            downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    out_channels*self.expansion,
                    kernel_size=1,
                    stride=stride,
                    bias=False
                ),
                nn.BatchNorm2d(out_channels * self.expansion),
            )
        layers = []
        layers.append(
            block(
                self.in_channels, out_channels, stride, self.expansion, downsample
            )
        )
        self.in_channels = out_channels * self.expansion
        for i in range(1, blocks):
            layers.append(block(
                self.in_channels,
                out_channels,
                expansion=self.expansion
            ))
        return nn.Sequential(*layers)
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [4]:
# set parameters 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
target_attribute = "Smiling"
sens_attribute = "Male"
nominal = False
num_epochs = 20 
lr_theta = 1e-3 
lr_z = 1e-1 
epsilon = 1e-2 

In [5]:
train_dataset = MyDataset(targ=target_attribute, sens_attr=sens_attribute, dataset_type='train')
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
sens_train, y_train = train_dataset.get_values()
flag  = get_flag(sens_train, y_train) 
sens_train = torch.Tensor(list(sens_train))
y_train = torch.Tensor(list(y_train))

test_dataset = MyDataset(targ=target_attribute, sens_attr=sens_attribute, dataset_type='test')
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True, drop_last=True)
sens_test, y_test = test_dataset.get_values()

  0%|          | 185344/243346528 [00:00<02:16, 1777641.02it/s]

243347456it [01:33, 2604190.61it/s]                               


Extracting ./data/lfw-py/lfw-funneled.tgz to ./data/lfw-py


66560it [00:00, 2296037.26it/s]          
95232it [00:00, 3449326.07it/s]          


100%|██████████| 9525/9525 [00:49<00:00, 192.68it/s]


Using downloaded and verified file: ./data/lfw-py/lfw-funneled.tgz
Extracting ./data/lfw-py/lfw-funneled.tgz to ./data/lfw-py


28672it [00:00, 1855792.79it/s]          

Using downloaded and verified file: ./data/lfw-py/lfw-names.txt



100%|██████████| 2708/2708 [00:14<00:00, 183.32it/s]


In [ ]:
z0_train = torch.tensor([0 for i in range(len(y_train))])
z0_train = z0_train.to(device)
z0_train = proj_z_unstr(z0_train, sens_train, y_train, flag, epsilon)
model = ResNet(img_channels=3, num_layers=18, block=BasicBlock, num_classes=1)
model = model.to(device)
opt = optim.Adam(model.parameters(), lr=lr_theta)
for _ in range(num_epochs): 
    train_err, train_loss, z = epoch_manual_unstr(train_loader, model, z0_train, sens_train, y_train, flag, device, epsilon=epsilon, lr_z=lr_z, opt=opt)
    z0_train = z

In [ ]:
# Evaluate Model
model.eval()
results_fairness = metrics_unstr(test_loader, model, flag, device)